## Solutions for Unit 4 Actions Part 1

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionExercise4-6">Solution Exercise 4.6</a>

## Solution Exercise 4.6 <p id="SolutionExercise4-6"></p>

<p style="background:#EE9023;color:white;">**Exercise 4.6**</p>

For this exercise, we will assume that our package is called **exercise_46**, our launch file is called **move_drone.launch**, and our C++ file is called **move_drone.cpp**.

First of all, you have to make sure that the Action Server is up and running. Otherwise, you won't be able to complete this exercise. For that, you can execute the following command:

In [ ]:
rostopic list | grep ardrone_action_server

If the Action Server is running, you will see this:

<img src="../img/ardrone_as_sol.png" width="600" />

If you don't get this output, it means that your Action Server is not running, so you will have to launch it. For that, just execute the following command:

In [ ]:
roslaunch ardrone_as action_server.launch

When you are sure that the Action Server is running, you can then proceed with the Exercise. Below you can check a possible solution to this Exercise.

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: move_drone.launch** </p>

In [ ]:
<launch>
    <node pkg="exercise_46" type="move_drone" name="drone_action_client" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: move_drone.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**C++ File: move_drone.cpp** </p>

In [ ]:
#include <ros/ros.h>
#include <ardrone_as/ArdroneAction.h> // Note: "Action" is appended
#include <actionlib/client/simple_action_client.h>
#include <geometry_msgs/Twist.h>
#include <std_msgs/Empty.h>

int nImage = 0;

void doneCb(const actionlib::SimpleClientGoalState& state,
            const ardrone_as::ArdroneResultConstPtr& result)
{
  ROS_INFO("[State Result]: %s", state.toString().c_str());
  ROS_INFO("The Action has been completed");
  //ros::shutdown();
}

// Called once when the goal becomes active
void activeCb()
{
  ROS_INFO("Goal just went active");
}

void feedbackCb(const ardrone_as::ArdroneFeedbackConstPtr& feedback)
{
  ROS_INFO("[Feedback] image n.%d received", nImage);
  ++nImage;
}

int main(int argc, char** argv)
{
  ros::init(argc, argv, "drone_action_client");
  ros::NodeHandle nh;
  
  actionlib::SimpleActionClient<ardrone_as::ArdroneAction> client("ardrone_action_server", true);
  client.waitForServer();
    
  ros::Publisher move = nh.advertise<geometry_msgs::Twist>("/cmd_vel", 1000);
  geometry_msgs::Twist move_msg;
  ros::Publisher takeoff = nh.advertise<std_msgs::Empty>("/drone/takeoff", 1000);
  std_msgs::Empty takeoff_msg;
  ros::Publisher land = nh.advertise<std_msgs::Empty>("/drone/land", 1000);
  std_msgs::Empty land_msg;

  ardrone_as::ArdroneGoal goal;
  goal.nseconds = 10;
  
  client.sendGoal(goal, &doneCb, &activeCb, &feedbackCb);
  //client.waitForResult();
  
  ros::Rate loop_rate(1);
  actionlib::SimpleClientGoalState state_result = client.getState();
  ROS_INFO("[State Result]: %s", state_result.toString().c_str());
  
  int i = 0;
  while (i < 4)
  {
      takeoff.publish(takeoff_msg);
      ROS_INFO("Taking Off Drone...");
      loop_rate.sleep();
      i++;
  }
    
  while ( state_result == actionlib::SimpleClientGoalState::ACTIVE || state_result == actionlib::SimpleClientGoalState::PENDING )
  {
    ROS_INFO("Moving drone around while waiting for the Server to give a result...");
    move_msg.linear.x = 1;
    move_msg.angular.z = 1;
    move.publish(move_msg);
    loop_rate.sleep();
    state_result = client.getState();
    ROS_INFO("[State Result]: %s", state_result.toString().c_str());
  }
  
  i = 0;
  while (i < 4)
  {
      move_msg.linear.x = 0;
      move_msg.angular.z = 0;
      move.publish(move_msg);
      land.publish(land_msg);
      ROS_INFO("Landing Drone...");
      loop_rate.sleep();
      i++;
  }
    
  return 0;
}

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END C++ File: move_drone.cpp** </p>

So, as you can see in the above code, the logic of the C++ code is quite simple:

* During the first 3 seconds, the drone will takeoff.
* Then, it will begin to move doing circles until the Action finishes
* When the Action finished, the drone will land

<img src="../img/move_drone_sol.gif" width="600" />